### - - 이전 내용 코드 시작 - - 

In [1]:
#코드 4-2 의존성 패키지 설치
!pip install ratsnlp

     |████████████████████████████████| 42 kB 1.1 MB/s 
     |████████████████████████████████| 2.8 MB 28.8 MB/s 
     |████████████████████████████████| 57 kB 7.2 MB/s 
     |████████████████████████████████| 806 kB 60.4 MB/s 
     |████████████████████████████████| 636 kB 62.5 MB/s 
     |████████████████████████████████| 398 kB 69.4 MB/s 
     |████████████████████████████████| 134 kB 73.0 MB/s 
     |████████████████████████████████| 829 kB 66.7 MB/s 
     |████████████████████████████████| 895 kB 31.8 MB/s 
     |████████████████████████████████| 3.3 MB 25.8 MB/s 
     |████████████████████████████████| 67 kB 3.2 MB/s 
     |████████████████████████████████| 1.1 MB 39.8 MB/s 
     |████████████████████████████████| 96 kB 5.2 MB/s 
     |████████████████████████████████| 271 kB 74.0 MB/s 
     |████████████████████████████████| 94 kB 4.4 MB/s 
     |████████████████████████████████| 144 kB 75.0 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sh

In [3]:
#코드 4-3 구글 드라이브와 연결
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [4]:
#코드 4-4 모델 환경 설정
import torch
from ratsnlp.nlpbook.classification import ClassificationTrainArguments
args = ClassificationTrainArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_corpus_name="nsmc",
    downstream_model_dir="/gdrive/My drive/nlpbook/checkpoint-doccls",
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    max_seq_length=128,
    epochs=3,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

In [5]:
#코드 4-5 랜덤 시드 고정
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


In [6]:
#코드 4-6 로거 설정
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='document-classification', downstream_corpus_name='nsmc', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My drive/nlpbook/checkpoint-doccls', max_seq_length=128, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, cpu_workers=2, fp16=False, tpu_cores=0)


In [7]:
#코드 4-7 말뭉치 내려받기
from Korpora import Korpora
Korpora.fetch(
    corpus_name=args.downstream_corpus_name,
    root_dir=args.downstream_corpus_root_dir,
    force_download=True,
)

[nsmc] download ratings_train.txt: 14.6MB [00:00, 225MB/s]
[nsmc] download ratings_test.txt: 4.90MB [00:00, 108MB/s]


In [8]:
#코드 4-8 토크나이저 준비
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

In [9]:
#코드 4-9 학습 데이터셋 구축
from ratsnlp.nlpbook.classification import NsmcCorpus, ClassificationDataset
corpus = NsmcCorpus()
train_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)

INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:loading train data... LOOKING AT /content/Korpora/nsmc/ratings_train.txt
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 35.723 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 아 더빙.. 진짜 짜증나네요 목소리
INFO:ratsnlp:tokens: [CLS] 아 더 ##빙 . . 진짜 짜증나네 ##요 목소리 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

In [10]:
train_dataset[0]

ClassificationFeatures(input_ids=[2, 2170, 832, 5045, 17, 17, 7992, 29734, 4040, 10720, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [11]:
#코드 4-10 학습 데이터 로더 구축
from torch.utils.data import DataLoader, RandomSampler
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers
)

In [12]:
#코드 4-11 평가용 데이터 로더 구축
from torch.utils.data import SequentialSampler
val_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="test",
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:loading test data... LOOKING AT /content/Korpora/nsmc/ratings_test.txt
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 11.817 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 굳 ㅋ
INFO:ratsnlp:tokens: [CLS] 굳 ㅋ [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

In [13]:
#코드 4-12 모델 초기화
from transformers import BertConfig, BertForSequenceClassification
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=corpus.num_labels,
)
model = BertForSequenceClassification.from_pretrained(
    args.pretrained_model_name,
    config=pretrained_model_config,
)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

In [14]:
#코드 4-13 TASK 정의
from ratsnlp.nlpbook.classification import ClassificationTask
task = ClassificationTask(model, args)

In [23]:
#코드 4-14 트레이너 정의
trainer = nlpbook.get_trainer(args)

OSError: ignored

In [24]:
#코드 4-15 학습 개시
trainer.fit(
    task,
    train_dataloader=train_dataloader,
    val_dataloaders=val_dataloader,
)

NameError: ignored

### - - 이전 내용 코드 끝 - - 

## <4-3 학습 마친 모델을 실전 투입하기>  

학습을 마친 문서 분류 모델을 ***인퍼런스(inference)*** 하는 과정을 실습해 보겠습니다. 인퍼런스란 학습을 마친 모델로 실제 과제를 수행하는 행위나 그 과정을 가리킵니다. 다시 말해 모델을 문서 분류라는 실전에 투입하는 것이죠.

### 영화 리뷰 감성 분석 웹 서비스 만들기
---
이번 실습에서는 앞 절에서 학습을 마친 문서 분류 모델을 가지고 웹 서비스를 만들어 보려고 합니다. 대강의 개념도는 다음 그림과 같습니다. 문장을 받아 해당 문장이 긍정인지 부정인지 답변하는 웹 서비스인데요, 문장을 토큰화한 뒤 모델 입력값으로 만들고 이를 모델에 입력해 [해당 문장이 긍정일 확률, 해당 문장이 부정일 확률]을 계산하게 만듭니다.  
이후 약간의 후처리 과정을 거쳐 응답하게 만드는 방식입니다.

<center><그림 4-8 웹 서비스의 역할></center>

<p align="center"><img src="https://i.imgur.com/I4lGm3J.jpg">  

<center>출처 : ratsgo's NLPBOOK</center>


웹 서비스란 네트워크에서 컴퓨터 간에 상호작용을 하기 위해 만들어진 소프트웨어 시스템입니다.  
이 장에서는 원격 사용자가 보낸 문장을 수신해 해당 문장이 긍정인지 부정인지 응답을 만들고, 이 응답을 원격 사용자에게 전달하는 웹 서비스를 코랩 환경에서 만듭니다.

<**1단계**> **코랩 노트북 초기화**  
이번 실습은 웹 브라우저에서 다음 주소(bit.ly/3leMWYr)에 접속하면 코랩 환경에서 수행할 수 있습니다. 이전 실습과 마찬가지로 코랩에서 **[내 드라이브에 복사]**를 진행합니다.  
다만 런타임 유형은 [None]으로 설정해 하드웨어 가속을 사용하지 않습니다.

<**2단계**> **환경 설정하기**   

우선 다음 코드를 실행해 의존성 있는 패키지를 설치합니다.

In [16]:
#코드 4-16 의존성 패키지 설치
!pip install ratsnlp

앞 절에서 학습한 모델의 체크포인트는 구글 드라이브에 저장해 두었으므로 다음 코드를 실행해 코랩 노트북과 자신의 구글 드라이브를 연동합니다.

In [17]:
#코드 4-17 구글 드라이브 연동
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


그리고 다음 코드는 실행해 각종 인자를 설정합니다.

In [25]:
#코드 4-18 인퍼런스 설정
from ratsnlp.nlpbook.classification import ClassificationDeployArguments
args = ClassificationDeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="/gdrive/My drive/nlpbook/checkpoint-doccls",
    max_seq_length=128,
)

Exception: ignored

각 인자의 역할과 내용은 다음과 같습니다.
>- **pretrainde_model_name:** 앞 절에서 적용한 pretrained_model_name(단, 해당 모델은 허깅페이스 라이브러리에 등록되어 있어야 합니다)
- **downstream_model_dir:** 앞 절에서 파인튜닝한 모델의 체크포인트 저장 위치(확장자가 ckpt인 파일이 하나 이상 있어야 합니다)
- **max_seq_length:** 토큰 기준 입력 문장 최대 길이. 아무것도 입력하지 않으면 128

<**3단계**> **토크나이저 및 모델 불러오기**   

다음 코드를 실행해 토크나이저를 초기화합니다.

In [26]:
#코드 4-19 토크나이저 로드
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

이어서 다음 코드는 앞 절에서 파인튜닝한 모델의 체크포인트를 읽어들입니다.

In [27]:
#코드 4-20 체크포인트 로드
import torch
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu"),
)

AttributeError: ignored

다음 코드는 앞 절 파인튜닝 때 사용한 **pretrained_model_name**에 해당하는 모델의 설정값들을 읽어들입니다. 이어서 코드 4-22를 실행하면 해당 설정값대로 BERT 모델을 초기화합니다.

In [28]:
#코드 4-21 BERT 설정 로드
from transformers import BertConfig
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt["state_dict"]["model.classifier.bias"].shape.numel(),
)

NameError: ignored

In [29]:
#코드 4-22 BERT 모델 초기화
from transformers import BertForSequenceClassification
model = BertForSequenceClassification(pretrained_model_config)

다음 코드는 초기화한 BERT 모델에 체크포인트(**fine_tuned_model_ckpt**)를 주입합니다. 이어서 코드 4-24를 실행하면 모델이 평가 모드로 전환됩니다.  
드롭아웃 등 학습 때만 사용하는 기법들을 무효화하는 역할을 합니다.

In [30]:
#코드 4-23 체크포인트 주입하기
model.load_state_dict({k.replace("mode.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})

NameError: ignored

In [31]:
#코드 4-24 평가 모드로 전환
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

<**4단계**> **모델 출력값 만들고 후처리하기**  

코드 4-25는 인퍼런스 과정을 정의한 함수입니다. 문장에 토큰화를 수행한 뒤 **input_ids, attention_mask, token_type_ids**를 만듭니다. 이들 입력값을 파이토치의 텐서 자료형으로 변환한 뒤 모델에 입력합니다. 모델 출력값(**outputs.logits**)은 소프트맥스 함수 적용 이전의 ***로짓(logit)*** 형태인데요, 여기에 소프트맥스 함수를 써서 모델 출력을 '[부정일 확률, 긍정일 확률]'로 바꿉니다.

마지막으로 모델 출력을 약간 후처리하여 예측 확률의 최댓값이 부정 위치일 때, 해당 문장이 '부정(negative)', 반대일 떄는 '긍정(positive)'이 되도록 **pred**값을 만듭니다.

In [32]:
#코드 4-25 인퍼런스 함수
def inference_fn(sentence):
    input = tokenizer(
        [sentence],
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True,
    )
    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})
        prob = outputs.logits.softmax(dim=1)
        positive_prob = round(prob[0][1].item(), 4)
        negative_prob = round(prob[0][0].item(), 4)
        pred = "긍정 (positive)" if torch.argmax(prob) == 1 else "부정 (negative)"
    return {
        'sentence': sentence,
        'prediction': pred,
        'positive_data': f"긍정 {positive_prob}",
        'negative_data': f"부정 {negative_prob}",
        'positive_width': f"{positive_prob * 100}%",
        'negative_width': f"{negative_prob * 100}%",
    }

이 코드에서 **positive_width, negative_width**는 웹 페이지에서 긍정/부정 막대의 길이를 조정하려는 것으로 크게 신경 쓰지 않아도 됩니다. 

<**5단계**> **웹 서비스 시작하기**   

앞 단계에서 정의한 인퍼런스 함수 **inference_fn**을 가지고 다음 코드를 실행하면 플라스크(flask)라는 파이썬 라이브러리의 도움을 받아 웹 서비스를 띄울 수 있습니다.

In [33]:
#코드 4-26 웹 서비스
from ratsnlp.nlpbook.classification import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app "ratsnlp.nlpbook.classification.deploy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://6a49-35-186-167-113.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [24/Mar/2022 08:36:13] "GET / HTTP/1.1" 200 -


그러면 다음 그림과 같은 출력 결과를 확인할 수 있는데요, 여기서 http://37e862e6897f.ngrok.io 라는 링크를 클릭합니다. 단, 주소는 코드 4-26을 실행할 때마다 바뀌므로 여러분이 실습할 때는 결과 화면에 보이는 주소로 접근해야 합니다. 그러면 웹 브라우저에서 다음과 같은 화면을 볼 수 있습니다. 

<center><그림 4-9 웹 서비스를 실행한 결과></center>

<p align="center"><img src="https://i.imgur.com/eLYbd3y.png">  
<p align="center"><img src="https://i.imgur.com/D2Kfvuk.png"> 
<p align="center"><img src="https://i.imgur.com/r7ira5x.png"> 

<center>출처 : ratsgo's NLPBOOK</center>

웹 페이지에서 '문장을 입력하세요' 부분에 문장(**ㅇㅈ**)을 입력하면 모델이 해당 문장의 극성(**긍정**)을 분류합니다. 긍정(0.8118)과 부정(0.1882) 확률도 출력됩니다.  
둘을 합치면 1이 됩니다. 

우리는 NSMC로 모델을 파인튜닝했으므로 모델은 입력된 문장이 영화 리뷰라고 생각하고 극성을 분석합니다.  
예를 들어 '졸립다'는 표현의 경우 침대 등 상품 리뷰에서는 긍정적인 극성을 가질 가능성이 큽니다. 하지만 영화 리뷰에서는 그 반대이므로 모델의 예측 결과 역시 이러한 경향을 따릅니다.  
즉, 학습 데이터의 특성에 따라 다른 모델이 도출된다는 이야기입니다.

#### 맺음말  
지금까지 우리는 문서 분류 모델을 만드는 과정을 실습했습니다. 실습 대상 말뭉치가 NSMC여서 해당 말뭉치로 학습한 모델은 영화 리뷰의 극성을 분류하는데요, 뉴스 본문과 카테고리(정치, 경제, 사회 등)로 짝지어진 말뭉치를 활용한다면 뉴스 카테고리를 분류하는 모델을 만들 수 있습니다. 그뿐만 아니라 허깅페이스 모델 허브에 등록만 돼 있으면 BERT 보다 좋은 성능을 가진 언어 모델을 활용할 수 있어 분류 성능을 더욱 끌어올릴 수 있습니다. 문서 분류 과제는 개체명 인식, 문서 생성 등 이후 소개할 과제의 기본이 되므로 잘 숙지해 두기 바랍니다.

#### <알아두면 좋아요>  

이번 장에서 소개한 내용과 관련해 추천할 자료가 2가지 있습니다. 하나는 구글에서 2018년 발표한 BERT 논문입니다. 제목과 링크는 다음과 같습니다.

- **Bidirectional Encoder Representations from Transformers:** arxiv.org/pdf/1810.04805.pdf

BERT는 트랜스포머의 인코더를 그대로 활용했다는 점에서 아키텍처상 독창성은 거의 없습니다. 이 때문에 처음에는 큰 주목을 받지 못했습니다. 하지만 저자들은 빈칸 맞히기(마스크 언어 모델)라는 프리트레인 태스크를 제안하고, 프리트레인을 마친 언어 모델로 문서 분류, 개체명 인식 등 다운스트림 태스크에 파인튜닝하는 프레임워크를 정립했습니다.   
BERT는 트랜스포머를 트랜스포 러닝에 본격적으로 적용한 첫 논문인 셈이죠.

BERT가 다양한 다운스트림 태스크에서 최고 성능을 기록하면서 입소문을 타기 시작해 세계적으로 유명한 논문이 되었습니다. 자연어 처리 분야 최고 학회 가운데 하나인 NAACL은 이러한 공로를 인정해 이 논문을 발표 이듬해인 2019년 최우수 논문으로 선정했습니다. 마스크 언어 모델을 제시하게 된 배경과 그 효과 등에 관해 저자들의 고민을 엿볼 수 있어서 읽어 볼 가치는 충분합니다.

BERT가 발표된 이후 비슷한 논문이 쏟아졌습니다. 이 가운데 주목할 기법은 페이스북이 2019년 발표한 RoBERTa입니다. 큰 틀에서는 BERT와 비슷하나 어휘 집합(vocabulary) 구축, 프리트래인 태스크, 학습 말뭉치 선정 등 디테일에서 차이를 보입니다. 성능이 좋아 최근에는 BERT 보다 널리 쓰이고 있습니다. 제목과 링크는 다음과 같습니다. 

- **RoBERTa: A Robustly Optimized BERT Pretraining Apporach:** arxiv.org/pdf/1907.11692.pdf